# 3. FETCHING TEAM RADIO (AUDIO) DATA

For now, we will be working on the 2023 season.

Track visualization practices were applied on Max Verstappen's performance in the 23-Bahrain Grand Prix. Let us understand the main source of team radio data and work on Verstappen's communication types, in this particular race.

### 3.1 REACHING THE JSON CONTENT

In [1]:
import fastf1
import os
import json
import requests
import urllib.request as request
import logging
from json import JSONDecodeError
import pandas as pd
import numpy as np
import speech_recognition as sr
from IPython.display import Audio 
import torch
import whisper
import tempfile
import soundfile as sf  
import random
import pickle
#import torchaudio

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
parent_dir = os.path.dirname(os.path.realpath("3. Fetching Team Radio.ipynb")) 
data_path = os.path.join(parent_dir, 'Data Objects')

The 2023 Emilia Romagna GP was cancelled due to flood.

In [2]:
race_schedule_2023 = fastf1.get_event_schedule(2023)
race_schedule_2023

req         WARNING 	DEFAULT CACHE ENABLED! (103.02 MB) C:\Users\sinem\AppData\Local\Temp\fastf1


,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Bahrain,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023-02-25,Pre-Season Testing,testing,Practice 1,2023-02-23 10:00:00+03:00,2023-02-23 07:00:00,...,Practice 3,2023-02-25 10:00:00+03:00,2023-02-25 07:00:00,None,NaT,NaT,None,NaT,NaT,True
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2023,2023-03-05,Bahrain Grand Prix,conventional,Practice 1,2023-03-03 14:30:00+03:00,2023-03-03 11:30:00,...,Practice 3,2023-03-04 14:30:00+03:00,2023-03-04 11:30:00,Qualifying,2023-03-04 18:00:00+03:00,2023-03-04 15:00:00,Race,2023-03-05 18:00:00+03:00,2023-03-05 15:00:00,True
2,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023,2023-03-19,Saudi Arabian Grand Prix,conventional,Practice 1,2023-03-17 16:30:00+03:00,2023-03-17 13:30:00,...,Practice 3,2023-03-18 16:30:00+03:00,2023-03-18 13:30:00,Qualifying,2023-03-18 20:00:00+03:00,2023-03-18 17:00:00,Race,2023-03-19 20:00:00+03:00,2023-03-19 17:00:00,True
3,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2023,2023-04-02,Australian Grand Prix,conventional,Practice 1,2023-03-31 12:30:00+10:00,2023-03-31 02:30:00,...,Practice 3,2023-04-01 12:30:00+10:00,2023-04-01 02:30:00,Qualifying,2023-04-01 16:00:00+10:00,2023-04-01 06:00:00,Race,2023-04-02 15:00:00+10:00,2023-04-02 05:00:00,True
4,4,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2023,2023-04-30,Azerbaijan Grand Prix,sprint_shootout,Practice 1,2023-04-28 13:30:00+04:00,2023-04-28 09:30:00,...,Sprint Shootout,2023-04-29 12:30:00+04:00,2023-04-29 08:30:00,Sprint,2023-04-29 17:30:00+04:00,2023-04-29 13:30:00,Race,2023-04-30 15:00:00+04:00,2023-04-30 11:00:00,True
5,5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07,Miami Grand Prix,conventional,Practice 1,2023-05-05 14:00:00-04:00,2023-05-05 18:00:00,...,Practice 3,2023-05-06 12:30:00-04:00,2023-05-06 16:30:00,Qualifying,2023-05-06 16:00:00-04:00,2023-05-06 20:00:00,Race,2023-05-07 15:30:00-04:00,2023-05-07 19:30:00,True
6,6,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2023,2023-05-28,Monaco Grand Prix,conventional,Practice 1,2023-05-26 13:30:00+02:00,2023-05-26 11:30:00,...,Practice 3,2023-05-27 12:30:00+02:00,2023-05-27 10:30:00,Qualifying,2023-05-27 16:00:00+02:00,2023-05-27 14:00:00,Race,2023-05-28 15:00:00+02:00,2023-05-28 13:00:00,True
7,7,Spain,Barcelona,FORMULA 1 AWS GRAN PREMIO DE ESPAÑA 2023,2023-06-04,Spanish Grand Prix,conventional,Practice 1,2023-06-02 13:30:00+02:00,2023-06-02 11:30:00,...,Practice 3,2023-06-03 12:30:00+02:00,2023-06-03 10:30:00,Qualifying,2023-06-03 16:00:00+02:00,2023-06-03 14:00:00,Race,2023-06-04 15:00:00+02:00,2023-06-04 13:00:00,True
8,8,Canada,Montréal,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2023,2023-06-18,Canadian Grand Prix,conventional,Practice 1,2023-06-16 13:30:00-04:00,2023-06-16 17:30:00,...,Practice 3,2023-06-17 12:30:00-04:00,2023-06-17 16:30:00,Qualifying,2023-06-17 16:00:00-04:00,2023-06-17 20:00:00,Race,2023-06-18 14:00:00-04:00,2023-06-18 18:00:00,True
9,9,Austria,Spielberg,FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2023,2023-07-02,Austrian Grand Prix,sprint_shootout,Practice 1,2023-06-30 13:30:00+02:00,2023-06-30 11:30:00,...,Sprint Shootout,2023-07-01 12:00:00+02:00,2023-07-01 10:00:00,Sprint,2023-07-01 16:30:00+02:00,2023-07-01 14:30:00,Race,2023-07-02 15:00:00+02:00,2023-07-02 13:00:00,True


In [4]:
race_schedule_2022 = fastf1.get_event_schedule(2022)
race_schedule_2022

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Spain,Spain,FORMULA 1 PRE-SEASON TRACK SESSION 2022,2022-02-25,Pre-Season Track Session,testing,Practice 1,2022-02-23 09:00:00+01:00,2022-02-23 08:00:00,...,Practice 3,2022-02-25 09:00:00+01:00,2022-02-25 08:00:00,None,NaT,NaT,None,NaT,NaT,False
1,0,Bahrain,Bahrain,FORMULA 1 ARAMCO PRE-SEASON TESTING 2022,2022-03-12,Pre-Season Test,testing,Practice 1,2022-03-10 10:00:00+03:00,2022-03-10 07:00:00,...,Practice 3,2022-03-12 10:00:00+03:00,2022-03-12 07:00:00,None,NaT,NaT,None,NaT,NaT,True
2,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022,2022-03-20,Bahrain Grand Prix,conventional,Practice 1,2022-03-18 15:00:00+03:00,2022-03-18 12:00:00,...,Practice 3,2022-03-19 15:00:00+03:00,2022-03-19 12:00:00,Qualifying,2022-03-19 18:00:00+03:00,2022-03-19 15:00:00,Race,2022-03-20 18:00:00+03:00,2022-03-20 15:00:00,True
3,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2022,2022-03-27,Saudi Arabian Grand Prix,conventional,Practice 1,2022-03-25 17:00:00+03:00,2022-03-25 14:00:00,...,Practice 3,2022-03-26 17:00:00+03:00,2022-03-26 14:00:00,Qualifying,2022-03-26 20:00:00+03:00,2022-03-26 17:00:00,Race,2022-03-27 20:00:00+03:00,2022-03-27 17:00:00,True
4,3,Australia,Melbourne,FORMULA 1 HEINEKEN AUSTRALIAN GRAND PRIX 2022,2022-04-10,Australian Grand Prix,conventional,Practice 1,2022-04-08 13:00:00+10:00,2022-04-08 03:00:00,...,Practice 3,2022-04-09 13:00:00+10:00,2022-04-09 03:00:00,Qualifying,2022-04-09 16:00:00+10:00,2022-04-09 06:00:00,Race,2022-04-10 15:00:00+10:00,2022-04-10 05:00:00,True
5,4,Italy,Imola,FORMULA 1 ROLEX GRAN PREMIO DEL MADE IN ITALY ...,2022-04-24,Emilia Romagna Grand Prix,sprint,Practice 1,2022-04-22 13:30:00+02:00,2022-04-22 11:30:00,...,Practice 2,2022-04-23 12:30:00+02:00,2022-04-23 10:30:00,Sprint,2022-04-23 16:30:00+02:00,2022-04-23 14:30:00,Race,2022-04-24 15:00:00+02:00,2022-04-24 13:00:00,True
6,5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2022,2022-05-08,Miami Grand Prix,conventional,Practice 1,2022-05-06 14:30:00-04:00,2022-05-06 18:30:00,...,Practice 3,2022-05-07 13:00:00-04:00,2022-05-07 17:00:00,Qualifying,2022-05-07 16:00:00-04:00,2022-05-07 20:00:00,Race,2022-05-08 15:30:00-04:00,2022-05-08 19:30:00,True
7,6,Spain,Barcelona,FORMULA 1 PIRELLI GRAN PREMIO DE ESPAÑA 2022,2022-05-22,Spanish Grand Prix,conventional,Practice 1,2022-05-20 14:00:00+02:00,2022-05-20 12:00:00,...,Practice 3,2022-05-21 13:00:00+02:00,2022-05-21 11:00:00,Qualifying,2022-05-21 16:00:00+02:00,2022-05-21 14:00:00,Race,2022-05-22 15:00:00+02:00,2022-05-22 13:00:00,True
8,7,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2022,2022-05-29,Monaco Grand Prix,conventional,Practice 1,2022-05-27 14:00:00+02:00,2022-05-27 12:00:00,...,Practice 3,2022-05-28 13:00:00+02:00,2022-05-28 11:00:00,Qualifying,2022-05-28 16:00:00+02:00,2022-05-28 14:00:00,Race,2022-05-29 15:00:00+02:00,2022-05-29 13:00:00,True
9,8,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2022,2022-06-12,Azerbaijan Grand Prix,conventional,Practice 1,2022-06-10 15:00:00+04:00,2022-06-10 11:00:00,...,Practice 3,2022-06-11 15:00:00+04:00,2022-06-11 11:00:00,Qualifying,2022-06-11 18:00:00+04:00,2022-06-11 14:00:00,Race,2022-06-12 15:00:00+04:00,2022-06-12 11:00:00,True


The link of the JSON file that keeps all audio data from a specific race:

livetiming.formula1.com/static/2023/2023-02-25_Bahrain_Grand_Prix/2023-02-25_Race/TeamRadio.jsonStream

In [11]:
def get_team_radio(location):
    logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
    url = location
    logging.info('Attempting to retrieve data from URL: %s', url)
    try:
        r = requests.get(location)
        r.encoding = 'utf-8-sig'
        out = json.loads(r.text)
        output = json.dumps(out, indent=4)
        logging.info('Data successfully retrieved from URL: %s', url)
        #return output
        return json.loads(output)
    except requests.RequestException as e:
        logging.error('Error retrieving data from URL: %s', url)
        logging.error(e)
        return None
    except JSONDecodeError:
        logging.warning('Failed to decode JSON data from URL: %s', url)
        return None

In [12]:
bahrain_location = "https://livetiming.formula1.com/static/2023/2023-03-05_Bahrain_Grand_Prix/2023-03-05_Race/TeamRadio.json"
data = get_team_radio(bahrain_location)
print("JSON data:", data) if data else print("Failed to retrieve JSON data.")


2024-06-10 13:08:01,681 - Attempting to retrieve data from URL: https://livetiming.formula1.com/static/2023/2023-03-05_Bahrain_Grand_Prix/2023-03-05_Race/TeamRadio.json
2024-06-10 13:08:02,349 - Data successfully retrieved from URL: https://livetiming.formula1.com/static/2023/2023-03-05_Bahrain_Grand_Prix/2023-03-05_Race/TeamRadio.json


JSON data: {'Captures': [{'Utc': '2023-03-05T14:20:25.563Z', 'RacingNumber': '20', 'Path': 'TeamRadio/KEVMAG01_20_20230305_141949.mp3'}, {'Utc': '2023-03-05T14:28:08.211Z', 'RacingNumber': '11', 'Path': 'TeamRadio/SERPER01_11_20230305_142737.mp3'}, {'Utc': '2023-03-05T14:58:31.7697978Z', 'RacingNumber': '20', 'Path': 'TeamRadio/KEVMAG01_20_20230305_145820.mp3'}, {'Utc': '2023-03-05T15:00:11.957Z', 'RacingNumber': '27', 'Path': 'TeamRadio/NICHUL01_27_20230305_145953.mp3'}, {'Utc': '2023-03-05T15:02:58.2510788Z', 'RacingNumber': '2', 'Path': 'TeamRadio/LOGSAR01_2_20230305_150237.mp3'}, {'Utc': '2023-03-05T15:07:58.261Z', 'RacingNumber': '27', 'Path': 'TeamRadio/NICHUL01_27_20230305_150739.mp3'}, {'Utc': '2023-03-05T15:09:04.8859816Z', 'RacingNumber': '11', 'Path': 'TeamRadio/SERPER01_11_20230305_150827.mp3'}, {'Utc': '2023-03-05T15:11:18.835Z', 'RacingNumber': '27', 'Path': 'TeamRadio/NICHUL01_27_20230305_151055.mp3'}, {'Utc': '2023-03-05T15:15:11.487Z', 'RacingNumber': '77', 'Path': 'Te

Let us automize the way of creating the link for a race other than Bahrain.

The formula goes as follows:

"https://livetiming.formula1.com/static/ **year** /**date**_**eventName**/**date**_Race/TeamRadio.json"

In [13]:
team_radio_links = {}
for i in race_schedule_2023[1:].index:
    date = race_schedule_2023.EventDate[i] #format %Y-%m-%d %H:%M:%S.%f
    event_name = race_schedule_2023.EventName[i].replace(" ", "_")
    country = race_schedule_2023.Country[i]
    
    team_radio_links[event_name] = "https://livetiming.formula1.com/static/" + date.strftime('%Y') + "/" + date.strftime('%Y-%m-%d') + "_" + event_name + "/" + date.strftime('%Y-%m-%d') + "_Race/TeamRadio.json"
    
print(team_radio_links)

{'Bahrain_Grand_Prix': 'https://livetiming.formula1.com/static/2023/2023-03-05_Bahrain_Grand_Prix/2023-03-05_Race/TeamRadio.json', 'Saudi_Arabian_Grand_Prix': 'https://livetiming.formula1.com/static/2023/2023-03-19_Saudi_Arabian_Grand_Prix/2023-03-19_Race/TeamRadio.json', 'Australian_Grand_Prix': 'https://livetiming.formula1.com/static/2023/2023-04-02_Australian_Grand_Prix/2023-04-02_Race/TeamRadio.json', 'Azerbaijan_Grand_Prix': 'https://livetiming.formula1.com/static/2023/2023-04-30_Azerbaijan_Grand_Prix/2023-04-30_Race/TeamRadio.json', 'Miami_Grand_Prix': 'https://livetiming.formula1.com/static/2023/2023-05-07_Miami_Grand_Prix/2023-05-07_Race/TeamRadio.json', 'Monaco_Grand_Prix': 'https://livetiming.formula1.com/static/2023/2023-05-28_Monaco_Grand_Prix/2023-05-28_Race/TeamRadio.json', 'Spanish_Grand_Prix': 'https://livetiming.formula1.com/static/2023/2023-06-04_Spanish_Grand_Prix/2023-06-04_Race/TeamRadio.json', 'Canadian_Grand_Prix': 'https://livetiming.formula1.com/static/2023/202

Apply the same procedure for the 2022 season.

In [21]:
team_radio_links22 = {}
for i in race_schedule_2022[2:].index: #First two are pre-season testings
    date = race_schedule_2022.EventDate[i] #format %Y-%m-%d %H:%M:%S.%f
    event_name = race_schedule_2022.EventName[i].replace(" ", "_")
    country = race_schedule_2022.Country[i]
    
    team_radio_links22[event_name] = "https://livetiming.formula1.com/static/" + date.strftime('%Y') + "/" + date.strftime('%Y-%m-%d') + "_" + event_name + "/" + date.strftime('%Y-%m-%d') + "_Race/TeamRadio.json"
    
print(team_radio_links22)

{'Bahrain_Grand_Prix': 'https://livetiming.formula1.com/static/2022/2022-03-20_Bahrain_Grand_Prix/2022-03-20_Race/TeamRadio.json', 'Saudi_Arabian_Grand_Prix': 'https://livetiming.formula1.com/static/2022/2022-03-27_Saudi_Arabian_Grand_Prix/2022-03-27_Race/TeamRadio.json', 'Australian_Grand_Prix': 'https://livetiming.formula1.com/static/2022/2022-04-10_Australian_Grand_Prix/2022-04-10_Race/TeamRadio.json', 'Emilia_Romagna_Grand_Prix': 'https://livetiming.formula1.com/static/2022/2022-04-24_Emilia_Romagna_Grand_Prix/2022-04-24_Race/TeamRadio.json', 'Miami_Grand_Prix': 'https://livetiming.formula1.com/static/2022/2022-05-08_Miami_Grand_Prix/2022-05-08_Race/TeamRadio.json', 'Spanish_Grand_Prix': 'https://livetiming.formula1.com/static/2022/2022-05-22_Spanish_Grand_Prix/2022-05-22_Race/TeamRadio.json', 'Monaco_Grand_Prix': 'https://livetiming.formula1.com/static/2022/2022-05-29_Monaco_Grand_Prix/2022-05-29_Race/TeamRadio.json', 'Azerbaijan_Grand_Prix': 'https://livetiming.formula1.com/stati

Now, all of the team radio JSON sources are in the **team_radio_links** dictionary.

Below, we run a small test to see all our links are correct and reachable. We can see that all the content is reached, successfully.

Now it is time to organize the data

In [ ]:
for i in team_radio_links.keys():
    print(i)
    data = get_team_radio(team_radio_links[i])
    print("") if data else print("ERROR! INDEX:",i)

In [ ]:
for i in team_radio_links22.keys():
    print(i)
    data = get_team_radio(team_radio_links22[i])
    print("") if data else print("ERROR! INDEX:",i)

Grouping the team radio so that all conversations from one driver is together.

In [23]:
def group_by_driver(data): 
    grouped_data = {}
    for item in data:
        racing_number = item['RacingNumber']
        if racing_number not in grouped_data:
            grouped_data[racing_number] = []
        grouped_data[racing_number].append(item)
        
    grouped_data = dict(sorted(grouped_data.items()))
    return grouped_data

In [24]:
def dict2df(dict):
    dfs = []
    for racing_number, data_list in dict.items():
        df = pd.DataFrame(data_list)
        dfs.append(df)
    
    result = pd.concat(dfs, keys=dict.keys())
    return result

In [ ]:
content = get_team_radio(team_radio_links['Bahrain_Grand_Prix'])
data = content['Captures']
grouped_data = group_by_driver(data)

In [ ]:
dict2df(grouped_data)

In [ ]:
dict2df(grouped_data).loc['1'] # Driver with racing number '1' (Max Verstappen)

Now we have a basic understanding of how our team radio data should be storaged and organized, we need to apply the same procedure for all countries.

We also convert the audio links into the appropriate form.


**link we need:** https://livetiming.formula1.com/static/2023/2023-03-05_Bahrain_Grand_Prix/2023-03-05_Race/TeamRadio/GUAZHO01_24_20230305_163627.mp3

**split**
https://livetiming.formula1.com/static/2023/2023-03-05_Bahrain_Grand_Prix/2023-03-05_Race/TeamRadio.json
->
https://livetiming.formula1.com/static/2023/2023-03-05_Bahrain_Grand_Prix/2023-03-05_Race/ + TeamRadio/GUAZHO01_24_20230305_163627.mp3

In [27]:
team_radio = {}
for k,v in team_radio_links.items():
    content = get_team_radio(team_radio_links[k])
    data = content['Captures']
    
    for capture in data:
        # CREATE NEW LINK
        path = capture['Path']
        path_new = v.split("TeamRadio.json",1)[0] + path #creating a new link        
        capture['Path New'] = path_new
        
        # CREATE TIMESTAMP
        ts = pd.Timestamp(capture['Utc']).round('1ms')
        ts_new = pd.Timestamp(ts.strftime("%Y-%m-%d %H:%M:%S.%f"))
        capture['Date'] = ts_new
    
    grouped_data = group_by_driver(data)
    team_radio[k] = dict2df(grouped_data)

2024-06-10 13:16:44,842 - Attempting to retrieve data from URL: https://livetiming.formula1.com/static/2023/2023-03-05_Bahrain_Grand_Prix/2023-03-05_Race/TeamRadio.json
2024-06-10 13:16:45,228 - Data successfully retrieved from URL: https://livetiming.formula1.com/static/2023/2023-03-05_Bahrain_Grand_Prix/2023-03-05_Race/TeamRadio.json
2024-06-10 13:16:45,249 - Attempting to retrieve data from URL: https://livetiming.formula1.com/static/2023/2023-03-19_Saudi_Arabian_Grand_Prix/2023-03-19_Race/TeamRadio.json
2024-06-10 13:16:45,779 - Data successfully retrieved from URL: https://livetiming.formula1.com/static/2023/2023-03-19_Saudi_Arabian_Grand_Prix/2023-03-19_Race/TeamRadio.json
2024-06-10 13:16:45,812 - Attempting to retrieve data from URL: https://livetiming.formula1.com/static/2023/2023-04-02_Australian_Grand_Prix/2023-04-02_Race/TeamRadio.json
2024-06-10 13:16:46,347 - Data successfully retrieved from URL: https://livetiming.formula1.com/static/2023/2023-04-02_Australian_Grand_Prix

In [25]:
team_radio22 = {}
for k,v in team_radio_links22.items():
    content = get_team_radio(team_radio_links22[k])
    data = content['Captures']
    
    for capture in data:
        # CREATE NEW LINK
        path = capture['Path']
        path_new = v.split("TeamRadio.json",1)[0] + path #creating a new link        
        capture['Path New'] = path_new
        
        # CREATE TIMESTAMP
        ts = pd.Timestamp(capture['Utc']).round('1ms')
        ts_new = pd.Timestamp(ts.strftime("%Y-%m-%d %H:%M:%S.%f"))
        capture['Date'] = ts_new
    
    grouped_data = group_by_driver(data)
    team_radio22[k] = dict2df(grouped_data)

2024-06-10 13:15:51,719 - Attempting to retrieve data from URL: https://livetiming.formula1.com/static/2022/2022-03-20_Bahrain_Grand_Prix/2022-03-20_Race/TeamRadio.json
2024-06-10 13:15:52,074 - Data successfully retrieved from URL: https://livetiming.formula1.com/static/2022/2022-03-20_Bahrain_Grand_Prix/2022-03-20_Race/TeamRadio.json
2024-06-10 13:15:52,126 - Attempting to retrieve data from URL: https://livetiming.formula1.com/static/2022/2022-03-27_Saudi_Arabian_Grand_Prix/2022-03-27_Race/TeamRadio.json
2024-06-10 13:15:52,466 - Data successfully retrieved from URL: https://livetiming.formula1.com/static/2022/2022-03-27_Saudi_Arabian_Grand_Prix/2022-03-27_Race/TeamRadio.json
2024-06-10 13:15:52,489 - Attempting to retrieve data from URL: https://livetiming.formula1.com/static/2022/2022-04-10_Australian_Grand_Prix/2022-04-10_Race/TeamRadio.json
2024-06-10 13:15:52,869 - Data successfully retrieved from URL: https://livetiming.formula1.com/static/2022/2022-04-10_Australian_Grand_Prix

In [28]:
print("COUNTRIES IN THE DICTIONARY\n",team_radio.keys())
print("\n\nCOLUMNS\n",team_radio['Saudi_Arabian_Grand_Prix'].columns)

COUNTRIES IN THE DICTIONARY
 dict_keys(['Bahrain_Grand_Prix', 'Saudi_Arabian_Grand_Prix', 'Australian_Grand_Prix', 'Azerbaijan_Grand_Prix', 'Miami_Grand_Prix', 'Monaco_Grand_Prix', 'Spanish_Grand_Prix', 'Canadian_Grand_Prix', 'Austrian_Grand_Prix', 'British_Grand_Prix', 'Hungarian_Grand_Prix', 'Belgian_Grand_Prix', 'Dutch_Grand_Prix', 'Italian_Grand_Prix', 'Singapore_Grand_Prix', 'Japanese_Grand_Prix', 'Qatar_Grand_Prix', 'United_States_Grand_Prix', 'Mexico_City_Grand_Prix', 'São_Paulo_Grand_Prix', 'Las_Vegas_Grand_Prix', 'Abu_Dhabi_Grand_Prix'])


COLUMNS
 Index(['Utc', 'RacingNumber', 'Path', 'Path New', 'Date'], dtype='object')


In [29]:
team_radio['Bahrain_Grand_Prix']

Utc RacingNumber  \
1  0  2023-03-05T15:19:06.8939905Z            1   
   1      2023-03-05T15:19:41.467Z            1   
   2      2023-03-05T15:35:51.622Z            1   
   3      2023-03-05T15:42:32.223Z            1   
   4      2023-03-05T15:50:57.699Z            1   
...                            ...          ...   
77 0      2023-03-05T15:15:11.487Z           77   
   1      2023-03-05T15:35:17.372Z           77   
   2      2023-03-05T16:20:27.776Z           77   
   3      2023-03-05T16:42:52.674Z           77   
81 0  2023-03-05T15:29:08.5257245Z           81   

                                           Path  \
1  0   TeamRadio/MAXVER01_1_20230305_151829.mp3   
   1   TeamRadio/MAXVER01_1_20230305_151925.mp3   
   2   TeamRadio/MAXVER01_1_20230305_153512.mp3   
   3   TeamRadio/MAXVER01_1_20230305_154159.mp3   
   4   TeamRadio/MAXVER01_1_20230305_155007.mp3   
...                                         ...   
77 0  TeamRadio/VALBOT01_77_20230305_151456.mp3   
   1  TeamRadio/VALBOT01_77_20230305_153449.mp3   
   2  TeamRadio/VALBOT01_77_20230305_162018.mp3   
   3  TeamRadio/VALBOT01_77_20230305_165208.mp3   
81 0  TeamRadio/OSCPIA01_81_20230305_152844.mp3   

                                               Path New  \
1  0  https://livetiming.formula1.com/static/2023/20...   
   1  https://livetiming.formula1.com/static/2023/20...   
   2  https://livetiming.formula1.com/static/2023/20...   
   3  https://livetiming.formula1.com/static/2023/20...   
   4  https://livetiming.formula1.com/static/2023/20...   
...                                                 ...   
77 0  https://livetiming.formula1.com/static/2023/20...   
   1  https://livetiming.formula1.com/static/2023/20...   
   2  https://livetiming.formula1.com/static/2023/20...   
   3  https://livetiming.formula1.com/static/2023/20...   
81 0  https://livetiming.formula1.com/static/2023/20...   

                        Date  
1  0 2023-03-05 15:19:06.894  
   1 2023-03-05 15:19:41.467  
   2 2023-03-05 15:35:51.622  
   3 2023-03-05 15:42:32.223  
   4 2023-03-05 15:50:57.699  
...                      ...  
77 0 2023-03-05 15:15:11.487  
   1 2023-03-05 15:35:17.372  
   2 2023-03-05 16:20:27.776  
   3 2023-03-05 16:42:52.674  
81 0 2023-03-05 15:29:08.526  

[84 rows x 5 columns]

In [30]:
team_radio22['Bahrain_Grand_Prix']

Utc RacingNumber  \
1  0  2022-03-20T14:19:09.733Z            1   
   1  2022-03-20T15:07:31.765Z            1   
   2  2022-03-20T15:09:34.552Z            1   
   3  2022-03-20T15:17:41.909Z            1   
   4   2022-03-20T15:21:45.65Z            1   
...                        ...          ...   
63 4  2022-03-20T15:49:09.011Z           63   
   5  2022-03-20T15:49:09.011Z           63   
   6  2022-03-20T16:07:25.571Z           63   
   7  2022-03-20T16:44:31.123Z           63   
77 0  2022-03-20T16:44:31.123Z           77   

                                           Path  \
1  0   TeamRadio/MAXVER01_1_20220320_141847.mp3   
   1   TeamRadio/MAXVER01_1_20220320_150702.mp3   
   2   TeamRadio/MAXVER01_1_20220320_150940.mp3   
   3   TeamRadio/MAXVER01_1_20220320_151709.mp3   
   4   TeamRadio/MAXVER01_1_20220320_152142.mp3   
...                                         ...   
63 4  TeamRadio/GEORUS01_63_20220320_154803.mp3   
   5  TeamRadio/GEORUS01_63_20220320_154820.mp3   
   6  TeamRadio/GEORUS01_63_20220320_160717.mp3   
   7  TeamRadio/GEORUS01_63_20220320_165701.mp3   
77 0  TeamRadio/VALBOT01_77_20220320_165809.mp3   

                                               Path New  \
1  0  https://livetiming.formula1.com/static/2022/20...   
   1  https://livetiming.formula1.com/static/2022/20...   
   2  https://livetiming.formula1.com/static/2022/20...   
   3  https://livetiming.formula1.com/static/2022/20...   
   4  https://livetiming.formula1.com/static/2022/20...   
...                                                 ...   
63 4  https://livetiming.formula1.com/static/2022/20...   
   5  https://livetiming.formula1.com/static/2022/20...   
   6  https://livetiming.formula1.com/static/2022/20...   
   7  https://livetiming.formula1.com/static/2022/20...   
77 0  https://livetiming.formula1.com/static/2022/20...   

                        Date  
1  0 2022-03-20 14:19:09.733  
   1 2022-03-20 15:07:31.765  
   2 2022-03-20 15:09:34.552  
   3 2022-03-20 15:17:41.909  
   4 2022-03-20 15:21:45.650  
...                      ...  
63 4 2022-03-20 15:49:09.011  
   5 2022-03-20 15:49:09.011  
   6 2022-03-20 16:07:25.571  
   7 2022-03-20 16:44:31.123  
77 0 2022-03-20 16:44:31.123  

[137 rows x 5 columns]

In [ ]:
team_radio['Mexico_City_Grand_Prix']

Now it is time to import the mp3 audio data.

### 3.2 IMPORTING AUDIO DATA

In [ ]:
audio_url = team_radio['Bahrain']['Path New'].iloc[3]
print(audio_url) #Max Verstappen's 8 seconds long radio.

Audio(audio_url, autoplay=True)

In [ ]:
def download_audio(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.content  # Return the audio data
    else:
        print("Failed to download audio")
        return None

Python's Speech Recognition module.
> https://pypi.org/project/SpeechRecognition/

Speech recognition engine/API support:
- CMU Sphinx (works offline)
- Google Speech Recognition
- Google Cloud Speech API
- Wit.ai
- Microsoft Azure Speech
- Microsoft Bing Voice Recognition (Deprecated)
- Houndify API
- IBM Speech to Text
- Snowboy Hotword Detection (works offline)
- Tensorflow
- Vosk API (works offline)
- OpenAI whisper (works offline)
- Whisper API

### 3.2.1 Google Speech Recognition Model

In [ ]:
def speech_to_text(audio_data, sample_rate, sample_width):
    recognizer = sr.Recognizer()
    # Converting the audio data to audio source
    audio_source = sr.AudioData(audio_data, sample_rate, sample_width)  #Sample_rate and sample_width

    # Speech to text conversion
    try:
        text = recognizer.recognize_google(audio_source)  #we are using Google Speech Recognition API
        return text
    except sr.UnknownValueError:
        print("Speech recognition could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Speech Recognition service; {e}")
        return None

In [ ]:
audio_data = download_audio(audio_url)
if audio_data:
    result = speech_to_text(audio_data, 48000, 4)
    if result:
        print("Text from audio:", result)

Ideas to solve this issue:

- Verify sample rate and width (from the original audio data source)
- To void background noises. (this might be the main issue)
    - Noise reduction 
    - Filters: Low-pass filters, high-pass filters, band-stop filters
    - Preprocessing :??
- Additional parameters: Sensitivity threshold, noise suppression settings, language?

### 3.2.2 OpenAI's Whisper Model

In [ ]:
def download_audio_new(url):
    response = requests.get(url)
    if response.status_code == 200:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            tmp_file.write(response.content)
            tmp_file_name = tmp_file.name
        audio_data, _ = sf.read(tmp_file_name, dtype='float32')  # Read audio file as waveform
        return audio_data
    else:
        print("Failed to download audio")
        return None

In [ ]:
def speech_to_text_whisper(url,model):
    audio_data = download_audio_new(url)
    results = model.transcribe(audio_data)
    return results['text']

In [ ]:
model = whisper.load_model("base.en")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)
options = whisper.DecodingOptions(language="en", without_timestamps=True)
options

In [ ]:
print(speech_to_text_whisper(audio_url,model))

In [ ]:
for country in team_radio.keys():
    print(f"{country:<20} : {len(team_radio[country]):<20}")

In [ ]:
country = random.sample(list(team_radio.keys()),k=1)[0]
index = random.randint(0, 55)

audio_url = team_radio[country]['Path New'].iloc[index]

print(speech_to_text_whisper(audio_url,model))
Audio(audio_url,autoplay=True)

Open AI's Whisper model also seems to be not working well.

### 3.4 SAVE DICTIONARY

We must save the team radio-timestamps dictionary to our file so that we can import and use in the next phases of our project.

In [ ]:
with open(os.path.join(data_path,'team_radio23.pkl'), 'wb') as f:
    pickle.dump(team_radio, f)

In [32]:
with open(os.path.join(data_path,'team_radio22.pkl'), 'wb') as f:
    pickle.dump(team_radio22, f)